## Train Normal and Small YOLOv8 with Tuned Hyperparameters/Augmentations

In [2]:
import os 
os.chdir('/home/jupyter/ee_tree_counting/Models/YOLOv8 Architecture/YOLOv8l')
import matplotlib.pyplot as plt
from ultralytics import YOLO
from IPython.display import display, Image
from PIL import Image
import pandas as pd
import numpy as np

In [10]:
import torch
torch.cuda.empty_cache()


In [7]:
!yolo detect train model=yolov8l.yaml data="/home/jupyter/ee_tree_counting/Data/Final Tree Dataset YOLO/data.yaml" workers=2 batch=4 device=0 epochs=100 patience=50 name=yolov8l_normal_tuned \
lr0=0.00253 lrf=0.00534 momentum=0.81527 weight_decay=0.00044 warmup_epochs=1.04965 warmup_momentum=0.5221 \
hsv_h=0.011 hsv_s=0.17527 hsv_v=0.16854 degrees=0.0 translate=0.02675 scale=0.09048 shear=0.0 perspective=0.0 \
flipud=0.0 fliplr=0.19722 bgr=0.0 mosaic=0.7526 mixup=0.0 copy_paste=0.0


New https://pypi.org/project/ultralytics/8.2.79 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (Tesla T4, 14918MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8l.yaml, data=/home/jupyter/ee_tree_counting/Data/Combined Dataset YOLO No Aug/data.yaml, epochs=100, time=None, patience=50, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=2, project=None, name=yolov8l_normal_tuned, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer

# Inference on Test Set

In [3]:
model = YOLO("/home/jupyter/ee_tree_counting/Models/YOLOv8 Architecture/YOLOv8l/runs/detect/yolov8l_normal_tuned/weights/best.pt")  

# Creating directory to store results
test_images_dir = '/home/jupyter/ee_tree_counting/Data/Final Tree Dataset YOLO/test/images'
output_dir = '/home/jupyter/ee_tree_counting/Data/Final Tree Dataset YOLO/test/YOLOv8l_normal_tuned Inference'
os.makedirs(output_dir, exist_ok=True)

# List of all test images
test_images = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith('.jpg')]
test_images.sort()

# Run inference on all test images
results = model(test_images)  

# Visualize and save the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Save results to disk
    im_rgb.save(os.path.join(output_dir, f"results_{i}.jpg"))

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 14.57 GiB total capacity; 47.12 MiB already allocated; 4.75 MiB free; 54.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

# Ground Truth

In [ ]:
label_dir = '/home/jupyter/ee_tree_counting/Data/Final Tree Dataset YOLO/test/labels'

# List of labels
labels = [os.path.join(label_dir, img) for img in os.listdir(label_dir) if img.endswith('.txt')]
labels.sort()

for idx, label in enumerate(labels):
    file = pd.read_csv(label, delimiter=' ', header=None)
    print(f'Ground Truth {idx}: {len(file)} trees')

In [9]:
predictions = [
    90, 7, 48, 46, 46, 46, 65, 70, 70, 70,
    69, 109, 109, 109, 109, 111, 99, 126, 174, 21,
    76, 76, 77, 76, 84, 84, 85, 85, 85, 82,
    82, 82, 81, 172, 115, 115, 115, 115, 77, 77,
    77, 77, 74, 49, 128, 126, 126, 128, 128, 92
]

ground_truth = [
    86, 7, 48, 48, 48, 48, 64, 69, 69, 69,
    69, 105, 114, 114, 114, 114, 109, 97, 175, 20,
    72, 72, 72, 72, 89, 89, 89, 89, 89, 77,
    77, 77, 77, 148, 112, 112, 112, 112, 72, 72,
    72, 72, 74, 50, 130, 130, 130, 130, 130, 89
]

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Image': [f'Image {i}' for i in range(len(predictions))],
    'Prediction': predictions,
    'Ground Truth': ground_truth
})

# Display the dataframe
print(results_df)

# Calculate Mean Absolute Error (MAE)
mae = (results_df['Prediction'] - results_df['Ground Truth']).abs().mean()

print(f"Mean Absolute Error (MAE): {mae:.2f}")


       Image  Prediction  Ground Truth
0    Image 0          90            86
1    Image 1           7             7
2    Image 2          48            48
3    Image 3          46            48
4    Image 4          46            48
5    Image 5          46            48
6    Image 6          65            64
7    Image 7          70            69
8    Image 8          70            69
9    Image 9          70            69
10  Image 10          69            69
11  Image 11         109           105
12  Image 12         109           114
13  Image 13         109           114
14  Image 14         109           114
15  Image 15         111           114
16  Image 16          99           109
17  Image 17         126            97
18  Image 18         174           175
19  Image 19          21            20
20  Image 20          76            72
21  Image 21          76            72
22  Image 22          77            72
23  Image 23          76            72
24  Image 24          84 

# Training and Tuning YOLOv8-small 

In [3]:
!yolo detect train model=yolov8s-small.yaml data="/home/jupyter/ee_tree_counting/Data/Final Tree Dataset YOLO/data.yaml" workers=2 batch=12 device=0 epochs=100 patience=50 name=yolov8s_small_tuned \
lr0=0.00253 lrf=0.00534 momentum=0.81527 weight_decay=0.00044 warmup_epochs=1.04965 warmup_momentum=0.5221 \
hsv_h=0.011 hsv_s=0.17527 hsv_v=0.16854 degrees=0.0 translate=0.02675 scale=0.09048 shear=0.0 perspective=0.0 \
flipud=0.0 fliplr=0.19722 bgr=0.0 mosaic=0.7526 mixup=0.0 copy_paste=0.0

New https://pypi.org/project/ultralytics/8.2.69 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (Tesla T4, 14918MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8s-small.yaml, data=/home/jupyter/ee_tree_counting/Data/Combined Dataset YOLO No Aug/data.yaml, epochs=100, time=None, patience=50, batch=12, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=2, project=None, name=yolov8s_small_tuned, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_

# Inference on Test Set

In [ ]:
model = YOLO("/home/jupyter/ee_tree_counting/Models/YOLOv8 Architecture/YOLOv8s/runs/detect/yolov8s_small_tuned/weights/best.pt")  

# Creating directory to store results
test_images_dir = '/home/jupyter/ee_tree_counting/Data/Final Tree Dataset YOLO/test/images'
output_dir = '/home/jupyter/ee_tree_counting/Data/Final Tree Dataset YOLO/test/YOLOv8s_small_tuned Inference'
os.makedirs(output_dir, exist_ok=True)

# List of all test images
test_images = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith('.jpg')]
test_images.sort()

# Run inference on all test images
results = model(test_images)  

# Visualize and save the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Save results to disk
    im_rgb.save(os.path.join(output_dir, f"results_{i}.jpg"))

In [8]:
predictions = [
    89, 6, 49, 49, 49, 49, 68, 67, 67, 67,
    68, 106, 113, 113, 113, 114, 98, 99, 178, 21,
    77, 77, 77, 77, 86, 86, 87, 87, 87, 78,
    78, 78, 79, 121, 116, 116, 116, 116, 74, 75,
    74, 74, 73, 49, 131, 131, 131, 131, 131, 95
]

ground_truth = [
    86, 7, 48, 48, 48, 48, 64, 69, 69, 69,
    69, 105, 114, 114, 114, 114, 109, 97, 175, 20,
    72, 72, 72, 72, 89, 89, 89, 89, 89, 77,
    77, 77, 77, 148, 112, 112, 112, 112, 72, 72,
    72, 72, 74, 50, 130, 130, 130, 130, 130, 89
]

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Image': [f'Image {i}' for i in range(len(predictions))],
    'Prediction': predictions,
    'Ground Truth': ground_truth
})

# Display the dataframe
print(results_df)

# Calculate Mean Absolute Error (MAE)
mae = (results_df['Prediction'] - results_df['Ground Truth']).abs().mean()

print(f"Mean Absolute Error (MAE): {mae:.2f}")

       Image  Prediction  Ground Truth
0    Image 0          89            86
1    Image 1           6             7
2    Image 2          49            48
3    Image 3          49            48
4    Image 4          49            48
5    Image 5          49            48
6    Image 6          68            64
7    Image 7          67            69
8    Image 8          67            69
9    Image 9          67            69
10  Image 10          68            69
11  Image 11         106           105
12  Image 12         113           114
13  Image 13         113           114
14  Image 14         113           114
15  Image 15         114           114
16  Image 16          98           109
17  Image 17          99            97
18  Image 18         178           175
19  Image 19          21            20
20  Image 20          77            72
21  Image 21          77            72
22  Image 22          77            72
23  Image 23          77            72
24  Image 24          86 

## YOLOv8s-small with it's own hyperparameters but same augmentation

In [2]:
!yolo detect train model=yolov8s-small.yaml data="/home/jupyter/ee_tree_counting/Data/Final Tree Dataset YOLO/data.yaml" workers=2 batch=12 device=0 epochs=100 patience=50 name=yolov8s_small_tuned_parameters_same_aug \
lr0=0.00233 lrf=0.01158 momentum=0.82094 weight_decay=0.00073 warmup_epochs=3.72712 warmup_momentum=0.48724 \
hsv_h=0.011 hsv_s=0.17527 hsv_v=0.16854 degrees=0.0 translate=0.02675 scale=0.09048 shear=0.0 perspective=0.0 \
flipud=0.0 fliplr=0.19722 bgr=0.0 mosaic=0.7526 mixup=0.0 copy_paste=0.0

New https://pypi.org/project/ultralytics/8.2.70 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (Tesla T4, 14918MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8s-small.yaml, data=/home/jupyter/ee_tree_counting/Data/Combined Dataset YOLO No Aug/data.yaml, epochs=100, time=None, patience=50, batch=12, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=2, project=None, name=yolov8s_small_tuned_parameters_same_aug, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, v

# Inference on Test Set

In [3]:
model = YOLO("/home/jupyter/ee_tree_counting/Models/YOLOv8 Architecture/YOLOv8s/runs/detect/yolov8s_small_tuned_parameters_same_aug/weights/best.pt")  

# Creating directory to store results
test_images_dir = '/home/jupyter/ee_tree_counting/Data/Final Tree Dataset YOLO/test/images'
output_dir = '/home/jupyter/ee_tree_counting/Data/Final Tree Dataset YOLO/test/YOLOv8s_small_tuned_parameters_same_aug Inference'
os.makedirs(output_dir, exist_ok=True)

# List of all test images
test_images = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith('.jpg')]
test_images.sort()

# Run inference on all test images
results = model(test_images)  

# Visualize and save the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Save results to disk
    im_rgb.save(os.path.join(output_dir, f"results_{i}.jpg"))


0: 640x640 87 trees, 17.1ms
1: 640x640 5 trees, 17.1ms
2: 640x640 51 trees, 17.1ms
3: 640x640 50 trees, 17.1ms
4: 640x640 50 trees, 17.1ms
5: 640x640 50 trees, 17.1ms
6: 640x640 65 trees, 17.1ms
7: 640x640 67 trees, 17.1ms
8: 640x640 67 trees, 17.1ms
9: 640x640 67 trees, 17.1ms
10: 640x640 67 trees, 17.1ms
11: 640x640 110 trees, 17.1ms
12: 640x640 108 trees, 17.1ms
13: 640x640 108 trees, 17.1ms
14: 640x640 108 trees, 17.1ms
15: 640x640 108 trees, 17.1ms
16: 640x640 102 trees, 17.1ms
17: 640x640 91 trees, 17.1ms
18: 640x640 174 trees, 17.1ms
19: 640x640 17 trees, 17.1ms
20: 640x640 76 trees, 17.1ms
21: 640x640 76 trees, 17.1ms
22: 640x640 76 trees, 17.1ms
23: 640x640 76 trees, 17.1ms
24: 640x640 91 trees, 17.1ms
25: 640x640 91 trees, 17.1ms
26: 640x640 91 trees, 17.1ms
27: 640x640 91 trees, 17.1ms
28: 640x640 91 trees, 17.1ms
29: 640x640 79 trees, 17.1ms
30: 640x640 79 trees, 17.1ms
31: 640x640 79 trees, 17.1ms
32: 640x640 79 trees, 17.1ms
33: 640x640 100 trees, 17.1ms
34: 640x640 119 

In [4]:
predictions = [
    87, 5, 51, 50, 50, 50, 65, 67, 67, 67,
    67, 110, 108, 108, 108, 108, 102, 91, 174, 17,
    76, 76, 76, 76, 91, 91, 91, 91, 91, 79,
    79, 79, 79, 100, 119, 120, 119, 119, 72, 71,
    72, 72, 74, 51, 131, 130, 130, 131, 131, 88
]

ground_truth = [
    86, 7, 48, 48, 48, 48, 64, 69, 69, 69,
    69, 105, 114, 114, 114, 114, 109, 97, 175, 20,
    72, 72, 72, 72, 89, 89, 89, 89, 89, 77,
    77, 77, 77, 148, 112, 112, 112, 112, 72, 72,
    72, 72, 74, 50, 130, 130, 130, 130, 130, 89
]

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Image': [f'Image {i}' for i in range(len(predictions))],
    'Prediction': predictions,
    'Ground Truth': ground_truth
})

# Display the dataframe
print(results_df)

# Calculate Mean Absolute Error (MAE)
mae = (results_df['Prediction'] - results_df['Ground Truth']).abs().mean()

print(f"Mean Absolute Error (MAE): {mae:.2f}")

       Image  Prediction  Ground Truth
0    Image 0          87            86
1    Image 1           5             7
2    Image 2          51            48
3    Image 3          50            48
4    Image 4          50            48
5    Image 5          50            48
6    Image 6          65            64
7    Image 7          67            69
8    Image 8          67            69
9    Image 9          67            69
10  Image 10          67            69
11  Image 11         110           105
12  Image 12         108           114
13  Image 13         108           114
14  Image 14         108           114
15  Image 15         108           114
16  Image 16         102           109
17  Image 17          91            97
18  Image 18         174           175
19  Image 19          17            20
20  Image 20          76            72
21  Image 21          76            72
22  Image 22          76            72
23  Image 23          76            72
24  Image 24          91 